### 2.5.3 例：記号代数

一般的な代数式を扱うには、難しすぎるので、<br />
ここではルールを設けて単純なもののみ扱います。

#### 多項式の算術演算

多項式に関する仕様
- 一元多項式（不定元が１つのみ）
- 多項式は項の和
- 項は係数、不定元の冪乗、係数と不定元の冪乗の積
- 係数は不定元と独立な代数式
- 演算は、加算と乗算のみ
- 組み合わせる２つの多項式は同じ不定元を持つ
- ジェネリック算術演算システムを拡張する形で多項式を追加する
- 多項式の加算は項ごとに実行<br />
同じ次数の項の場合は、足す項と足される項の係数の合計を係数同じ次数を持つ新しい項を作る<br />
片方にだけ項があって、もう一方のほうに同じ次数の項がない場合は、構築中の和の多項式にその項をそのまま追加
- 空の項リストを返すコンストラクタ the-empty-termlist を持つ
- 項リストに新しい項を追加するコンストラクタ adjoin-term を持つ
- 与えられた項リストが空かどうか調べる empty-termlist? という述語を持つ
- 項リストから最⼤次数の項を取り出すセレクタ first-term を持つ
- 数の項以外のすべての項を返すセレクタ rest-terms を持つ
- ジェネリック手続きaddとmulを持つ

項に関する仕様
- 与えられた次数と係数から項を構築するコンストラクタ make-term を持つ
- 項の次数を返すセレクタ order を持つ
- 項の係数を返すセレクタ coeff を持つ

多項式は、項リストとして表現します。<br />
強制型変換が必要かは不明です。（おそらく必要？）

#### 項リストの表現

 $x^{100} + 2 x^2 +1$  

を効率表現するように、
次数とその項の係数のリストを使って、

    ((100 1) (2 2) (0 1)) 
 
と表現し、高次から低次の順に並べられた項のリストとして表現します。

また、scheme-numberは練習問題2.78にあるように数値そのものとなるよう定義にします。

In [1]:
(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))
(define get-coercion get)
(define put-coercion put)

; タグつきデータのコンストラクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (attach-tag type-tag contents)
  (if (number? contents)
      contents
      (cons type-tag contents)))

; タグを返すセレクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (type-tag datum)
  (cond ((number? datum) 'scheme-number)
        ((pair? datum) (car datum))
        (else (error "Bad tagged datum: TYPE-TAG" datum))))

; データを返すセレクタ　練習問題2.78にあるsheme-numberは数値そのものとする
(define (contents datum)
  (cond ((number? datum) datum)
        ((pair? datum) (cdr datum))
        (else (error "Bad tagged datum: CONTENTS" datum))))

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (if (not (eq? type1 type2))
                    (let ((t1->t2 (get-coercion type1 type2))
                          (t2->t1 (get-coercion type2 type1)))
                      (cond (t1->t2
                             ;(apply-generic op (t1->t2 a1) a2))
                             (apply-generic op (t1->t2 a1 a2) a2))
                            (t2->t1
                             ;(apply-generic op a1 (t2->t1 a2)))
                             (apply-generic op a1 (t2->t1 a2 a1)))
                            (else (error "No method for these types" (list op type-tags)))
                            )
                      )
                    (error "Same types" (list op type-tags))
                    )
                )
              (error "No method for these types" (list op type-tags)))))
      )
    )

; ジェネリック算術演算手続き
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))
(define (=zero? x) (apply-generic '=zero? x))    ;追加手続き
(define (reverse-sign x) (apply-generic 'reverse-sign x))    ;追加手続き

; scheme-number
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put '=zero? '(scheme-number) (lambda (x) (= x 0)))    ;追加手続き
  (put 'reverse-sign '(scheme-number) (lambda (x) (* x -1)))    ;追加手続き
  (put 'make 'scheme-number (lambda (x) (tag x)))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [2]:
; scheme-numberの動作確認
(define a (make-scheme-number 2))
(define b (make-scheme-number 4))

(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")
(display (reverse-sign a))
(display "\n")

6
-2
8
1/2
-2


In [3]:
(define c (make-scheme-number 0))
(=zero? c)

#t

In [4]:
; polynomialの実装
(define (install-polynomial-package)
  ; 内部⼿続き
  ; poly の表現
  (define (make-poly variable term-list) (cons variable term-list))
  (define (variable p) (car p))
  (define (term-list p) (cdr p))
  ; 2.3.2 節の same-variable? と variable? ⼿続き
  (define (same-variable? v1 v2) (and (variable? v1) (variable? v2) (eq? v1 v2)))
  (define (variable? x) (symbol? x))
  ; 項と項リストの表現
  ; 下記の adjoin-term . . . coeff ⼿続き
  (define (adjoin-term term term-list)
    (if (=zero? (coeff term))
        term-list
        (cons term term-list)))
  (define (the-empty-termlist) '())
  (define (first-term term-list) (car term-list))
  (define (rest-terms term-list) (cdr term-list))
  (define (empty-termlist? term-list) (null? term-list))
  (define (make-term order coeff) (list order coeff))
  (define (order term) (car term))
  (define (coeff term) (cadr term))
  (define (add-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (add-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: ADD-POLY" (list p1 p2))))
  (define (mul-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (make-poly (variable p1)
                   (mul-terms (term-list p1) (term-list p2)))
        (error "Polys not in same var: MUL-POLY" (list p1 p2))))  
  ; add-poly/mul-polyが使う⼿続き
  (define (add-terms L1 L2)
    (cond ((empty-termlist? L1) L2)
          ((empty-termlist? L2) L1)
          (else
           (let ((t1 (first-term L1))
                 (t2 (first-term L2)))
             (cond ((> (order t1) (order t2))
                    (adjoin-term
                     t1 (add-terms (rest-terms L1) L2)))
                   ((< (order t1) (order t2))
                    (adjoin-term
                     t2 (add-terms L1 (rest-terms L2))))
                   (else
                    (adjoin-term
                     (make-term (order t1)
                                (add (coeff t1) (coeff t2)))
                     (add-terms (rest-terms L1)
                                (rest-terms L2)))))))))
  (define (mul-terms L1 L2)
    (if (empty-termlist? L1)
        (the-empty-termlist)
        (add-terms (mul-term-by-all-terms (first-term L1) L2)
                   (mul-terms (rest-terms L1) L2))))
  (define (mul-term-by-all-terms t1 L)
    (if (empty-termlist? L)
        (the-empty-termlist)
        (let ((t2 (first-term L)))
          (adjoin-term
           (make-term (+ (order t1) (order t2))
                      (mul (coeff t1) (coeff t2)))
           (mul-term-by-all-terms t1 (rest-terms L))))))
  ; 符号反転
  (define (reverse-sign-proc p)
        (make-poly (variable p)
                   (reverse-sign-terms (term-list p)))    
    )
  (define (reverse-sign-terms L1)
    (if (empty-termlist? L1) (the-empty-termlist)
        (let ((t1 (first-term L1)))
          (adjoin-term
           (make-term (order t1) (reverse-sign (coeff t1)))
           (reverse-sign-terms (rest-terms L1))))))
  ; 減算
  (define (sub-poly p1 p2)
    (if (same-variable? (variable p1) (variable p2))
        (add-poly p1 (reverse-sign-proc p2))
        (error "Polys not in same var: ADD-POLY" (list p1 p2))))
  (define (sub-terms t1 t2)
        (add-terms t1 (reverse-sign-terms t2))
    )
  (define (zero-terms L)
    (if (empty-termlist? L) #t
        (let ((t1 (first-term L)))
          (if (not (=zero? (coeff t1))) #f
              (zero-terms (rest-terms L))
          )
          )
        )
    )
  ; 強制型変換
  (define (scheme-number->polynomial n p)
    (tag (make-poly (variable (contents p)) (list (make-term 0 (contents n)))))
    )
  ; システムのほかの部分とのインターフェイス
  (define (tag p) (attach-tag 'polynomial p))
  (put 'add '(polynomial polynomial) (lambda (p1 p2) (tag (add-poly p1 p2))))
  (put 'mul '(polynomial polynomial) (lambda (p1 p2) (tag (mul-poly p1 p2))))
  ;(put '=zero? '(polynomial) (lambda (p) (or (eq? p '()) (=zero? p))))  ; 追加手続き
  (put '=zero? '(polynomial) (lambda (p) (zero-terms (term-list p))))  ; 追加手続き
  (put 'reverse-sign '(polynomial) (lambda (p) (tag (reverse-sign-proc p))))  ; 追加手続き
  (put 'sub '(polynomial polynomial) (lambda (p1 p2) (tag (sub-poly p1 p2))))  ; 追加手続き
  ; 追加手続き
  (put 'div '(polynomial polynomial) (lambda (p1 p2)
                                       (let ((ans (div-poly p1 p2)))
                                         ;ans
                                         (list (tag (car ans)) (tag (cadr ans)))
                                         )
                                       )
       )
  (put 'make 'polynomial (lambda (var terms) (tag (make-poly var terms))))
  (put-coercion  'scheme-number 'polynomial scheme-number->polynomial)
'done)


(define (make-polynomial var terms) ((get 'make 'polynomial) var terms))

(install-polynomial-package)

done

In [5]:
(define a (make-polynomial 'x '((2 1) (1 1) (0 1))))
(define b (make-polynomial 'x '((2 2) (1 -2) (0 5))))
(add a b)

(polynomial x (2 3) (1 -1) (0 6))

$(x^2 + x + 1)(2x^2 - 2x + 5)$  

$x^2 \times (2x^2 - 2x + 5) =  2x^4 -2x^3   + 5x^2$  
$x \times (2x^2 - 2x + 5) =          2 x^3   -2 x^2    + 5x$  
$1 \times (2x^2 - 2x + 5)    =                 2x^2    - 2x     +5$

$                              2x^4          +5x^2     +3x      + 5$


In [6]:
(mul a b)

(polynomial x (4 2) (2 5) (1 3) (0 5))

In [7]:
(define d (make-scheme-number -10))
(reverse-sign d)

10

In [8]:
(reverse-sign b)

(polynomial x (2 -2) (1 2) (0 -5))

In [9]:
(sub a b)

(polynomial x (2 -1) (1 3) (0 -4))

In [10]:
(define e (make-polynomial 'x '((2 0) (1 1) (0 0))))
(=zero? e)

#f

In [11]:
((get-coercion 'scheme-number 'polynomial) d a)

(polynomial x (0 -10))

In [12]:
a

(polynomial x (2 1) (1 1) (0 1))

In [13]:
(define c (make-polynomial 'y '((1 2) (0 2))))
c

(polynomial y (1 2) (0 2))

In [14]:
(define e (make-polynomial 'x (list (list 2 c) '(1 1) '(0 1))))
;(define e (make-polynomial 'x ((make-term 2 c) (make-term 1 1) (make-term 0 1))))
e

(polynomial x (2 (polynomial y (1 2) (0 2))) (1 1) (0 1))

In [15]:
(add e a)

(polynomial x (2 (polynomial y (1 2) (0 3))) (1 2) (0 2))

時間があったら試してみる

    polynamial
   ↑       ↑     
complex     
   ↑
rational
   ↑
scheme-number

時間があったら試してみる


#### 練習問題

- [練習問題2.87](../exercises/2.87.ipynb)
- [練習問題2.88](../exercises/2.88.ipynb)
- [練習問題2.89](../exercises/2.89.ipynb)
- [練習問題2.90](../exercises/2.90.ipynb)
- [練習問題2.91](../exercises/2.91.ipynb)

#### 記号代数の方の階層

#### 練習問題

- [練習問題2.92](../exercises/2.92.ipynb)


#### 拡張問題:有理関数

#### 練習問題

- [練習問題2.93](../exercises/2.93.ipynb)
- [練習問題2.94](../exercises/2.94.ipynb)
- [練習問題2.95](../exercises/2.95.ipynb)
- [練習問題2.96](../exercises/2.96.ipynb)
- [練習問題2.97](../exercises/2.97.ipynb)